<a href="https://colab.research.google.com/github/salarbalou/steadystate_analysis/blob/main/CQD_LAMP_thermal_profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import requests as req
import plotly.io as pio
import plotly.graph_objects as go
import re
import sympy as sy
from sympy import symbols, diff
from sympy.solvers import solve
from scipy.misc import derivative
import string as sr
pio.templates.default ='plotly_dark'
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
def findmb(x, y): # a simple function to find the slope and intecept of a line
  m,b = np.polyfit(x, y, deg= 1)
  return m,b

In [ ]:
#arduino code used to achieve this thermal profile is as shown below:
from google.colab import files
!wget 'https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/Optimize_T_LAMP.ino'
code = files.view('Optimize_T_LAMP.ino')

In [20]:
baseurl = 'https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/' # base url of the text file 
char = '.txt' # format of the file
process_description= '10-28-21-CQD-Colorimetric%20LAMP' #experiment was performed on 10-20-21 using CQD 10x/ dutycycle values change
text = req.get(baseurl+process_description+char).text # we download and store the text of each file.txt
data_path = re.compile(r'^([0-9.]+)[\s,]+([0-9.]+)+[\s,]+([-0-9.]+)+[\s,]+([0-9]+)', re.MULTILINE) # we grab the desired text within the text file (all data except for space and comma)
data_list = data_path.findall(text) # this is a list of quadruple strings: each quadruple in this list have abcd string in it. 
data = []
for quadruple in data_list:
    eachquadruple = [] # this list here stores the float version of abcd in each quadruple: eachquadruple is a list of four float values
    for abcd in quadruple:
          eachquadruple.append(float(abcd))
    data.append((eachquadruple[0], eachquadruple[1], eachquadruple[2], eachquadruple[3])) # we append all eachquadruples in data
final_data = np.array(data) # here we make an array of the list: final_data is a matrix with 183 rows and 4 columns (t, T, I, dc)
time = final_data[:, 0][1:]
temperature = final_data[:, 1][1:]
temperature1 = temperature[22:]
equ_temp = np.max(temperature, axis=0) # as mentioned, we assume the max T is the equ_T
current = np.sum(-final_data[:, 2][24:162], axis = 0)/138 #Amps
dutycycle = final_data[:, 3][23:162]
beam_height = 0.5 #cm
beam_width = 0.2 #cm
voltage_value = np.linspace(6, 7.5, num = 230) # max voltage is 7.5V, min voltage required for laser to turn on is 6V. if we polyfit V with dutycyle(0:250), find the relation of V and dc
dutycycle_value  = np.arange(20, 250,1)
m, b = findmb(dutycycle_value, voltage_value) 
realvoltage = np.sum(m*dutycycle+b)/dutycycle.size #V
laser_power_density = np.array(((realvoltage*current)/(beam_height*beam_width))) #W/cm^2
dt = np.gradient(time)
dT = np.gradient(temperature)
diffTt = np.sqrt((dT/dt)**2)
fig = make_subplots()
fig.add_scatter(x = time, y = temperature, mode = 'markers+lines', marker_color = 'red', row = 1, col = 1, name = 'Temperature vs time')
fig.update_xaxes(title_text = 'time (seconds)', nticks = 80, row = 1, col = 1)
fig.update_yaxes(title_text = 'Temperature', nticks=20, row = 1, col = 1)
fig.update_layout(height = 800, width = 2400)
fig.show()

In [ ]:
laser_power_density

array(35.16137557)